# Numba `parallel=True` a `prange`
Minule jsme si už ukázali, že numba nabízí automatický paralelismus (pokud to jde).

Našemu kódu můžeme pomoci a říct mu, kde má paralelizovat for cyklus pomocí `prange` (paralel range).

In [ ]:
# možná bude třeba
#!pip install --upgrade numba

In [ ]:
from numba import jit, prange
import numpy as np

@jit(nopython = True, parallel=True)
def test(x):
    n = x.shape[0]
    a = np.sin(x)
    b = np.cos(a * a)
    acc = 0
    for i in prange(n - 2):
        for j in prange(n - 1):
            acc += b[i] + b[j + 1]
    return acc


In [ ]:
test(np.arange(10))


Numba umožňuje inspekci paralelizace kódu pomocí `.parallel_diagnostics()`.

In [ ]:

test.parallel_diagnostics(level=4)

## Dot product z minula

In [ ]:
import numba
import math

@numba.jit(signature_or_function='float64(float64[:])',
           nopython=True,
           parallel=True,
           fastmath=True,
           locals={'result': numba.float64})
def my_norm_numba(a):
    result = 0
    for i in range(len(a)):
        result += a[i] * a[i]
    return math.sqrt(result)

In [ ]:
import numpy as np
x = np.random.rand(int(1e7*8))
y1 = my_norm_numba(x)
y2 = np.linalg.norm(x)
print(y1,y2)

In [ ]:
%timeit _ = my_norm_numba(x)

In [ ]:
%timeit _ = np.linalg.norm(x)

## Přidáme paralelizaci

In [ ]:
import numba
import math

@numba.jit(signature_or_function='float64(float64[:])',
           nopython=True,
           parallel=True,
           fastmath=True,
           locals={'result': numba.float64})
def my_norm_numba_parallel(a):
    result = 0
    for i in numba.prange(len(a)):
        result += a[i] * a[i]
    return math.sqrt(result)

In [ ]:
%timeit _ = my_norm_numba_parallel(x)

Kolik vláken vlastně Numba použije?

In [ ]:
numba.get_num_threads()


Můžeme to zkusit změnit, ale numba dovolí pouze tokik kolik je logických jader.

In [ ]:
numba.set_num_threads(2)

In [ ]:
%timeit _ = my_norm_numba_parallel(x)